# Demo for how GoodData Python SDK can be used inside notebooks

[GoodData Python SDK](https://www.gooddata.com/docs/cloud/api-and-sdk/python-sdk/) is an open-source set of libraries derived from GoodData OpenAPI specification.
It provides an additional layer of abstraction on top of the raw APIs.
Besides gooddata-sdk (core) library, we provide gooddata-pandas exposing report results as data frames.

## Import libraries

In [ ]:
from pathlib import Path
import pandas as pd 
from gooddata_sdk import GoodDataSdk
from gooddata_pandas import GoodPandas

## Init GoodData

In [ ]:
current_profile = "demo_cicd_dev"
workspace_id = "cicd_demo_development_motherduck"
profiles= Path.home() / ".gooddata" / "profiles.yaml"

gd_sdk = GoodDataSdk.create_from_profile(profile=current_profile, profiles_path=profiles)
gd_pandas = GoodPandas.create_from_profile(profile=current_profile, profiles_path=profiles)
gd_frames = gd_pandas.data_frames(workspace_id)

## List Metrics in a Workspace

In [ ]:
metrics = gd_sdk.catalog_workspace_content.get_metrics_catalog(workspace_id)
metrics_list = [{"id": m.id, "title": m.title} for m in metrics]
df = pd.DataFrame(metrics_list)
df

## Execute Already Defined Report

In [ ]:
insight_df = gd_frames.for_visualization('merged_prs_month')
insight_df

## Execute a Custom Report

In [ ]:
from gooddata_sdk import ExecutionDefinition, Attribute, SimpleMetric, ObjId, RelativeDateFilter, TableDimension
exec_def = ExecutionDefinition(
    attributes=[
        Attribute(local_id="created_at", label="created_at.month"),
        Attribute(local_id="repo_name", label="repo_name"),
    ],
    metrics=[
        SimpleMetric(local_id="count_commits", item=ObjId(id="commit_id", type="label"), aggregation="COUNT"),
        SimpleMetric(local_id="count_pull_requests", item=ObjId(id="count_pull_requests", type="metric")),
    ],
    filters=[RelativeDateFilter(dataset=ObjId(id="created_at", type="dataset"), granularity="YEAR", from_shift=-2, to_shift=0)],
    dimensions=[TableDimension(item_ids=["created_at"]), TableDimension(item_ids=["repo_name", "measureGroup"])],
)
df, df_metadata = gd_frames.for_exec_def(exec_def=exec_def)

# use result ID from computation above and generate dataframe just from it
df_from_result_id, df_metadata_from_result_id = gd_frames.for_exec_result_id(
    result_id=df_metadata.execution_response.result_id,
)
df_from_result_id

## Conclusion

This notebook is just a quick onboarding into how to leverage GoodData Python SDK libraries in a notebook.
I recommend you to check our [DOC](https://www.gooddata.com/docs/cloud/api-and-sdk/python-sdk/) for more details.
I tested the same approach in DeepNote and it worked.
There is no reason why it should work in any other notebook.